In [14]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
from datetime import datetime
import os
import random
import pandas as pd
from skimage.io import imread
from skimage.exposure import adjust_gamma
from skimage.exposure import is_low_contrast

In [15]:
!mkdir output

A subdirectory or file output already exists.


## Stokes-Einstein relation

In [16]:
def stokes_einstein(r, t, T_Celsius, eta, pixel_size):
    # Boltzmann's constant
    k_B = 1.38e-23

    T_Kelvin = T_Celsius + 273.15

    #r = r_pixel*pixel_size

    # Calculate diffusion coefficient using Stokes-Einstein relation
    D = (k_B * T_Kelvin) / (6 * np.pi * eta * r)
    variance = 2 * D * t
    sd = np.sqrt(variance)
    #print(sd / pixel_size)
    return sd / pixel_size

## Multi Particle Brownian Motion Simulation

In [17]:
def multi_brownian_motion(num_particles, num_steps, radius, xy_range, t, T_Celsius, eta, pixel_size):
    sigmas = stokes_einstein(radius, t, T_Celsius, eta, pixel_size)
    
    positions = np.zeros((num_steps, num_particles, 2))
    positions[0, :, 0] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    positions[0, :, 1] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    
    for step in range(1, num_steps):
        move = np.column_stack((np.random.normal(0, sigmas, num_particles), 
                                 np.random.normal(0, sigmas, num_particles)))
        positions[step] = positions[step - 1] + move
    return positions

In [18]:
# def multi_brownian_motion(num_particles, num_steps, sigma=1.6, xy_range=(0, 100)):
#     positions = np.zeros((num_steps, num_particles, 2))
#     positions[0, :, 0] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
#     positions[0, :, 1] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    
#     for step in range(1, num_steps):
#         positions[step] = positions[step - 1] + np.random.normal(0, sigma, (num_particles, 2))
#     return positions

## Model Parameters

In [19]:
num_particles = 22
num_steps = 50

resolution = (2048, 2048)

#in pixels
# radius = np.random.normal(5, 1, num_particles)
# radius = radius.round(4)
# radius /= 2 

sizesample = pd.read_excel('particlesize.xlsx')['diameter']/2
radius = np.array(random.choices(sizesample, k=num_particles))

d = 99e-9
r = d / 2
r_list = radius/sizesample.median()*r
# grayscale_value = [random.randint(50, 255) for _ in range(num_particles)]
# fill_color = [g for g in grayscale_value]

# Tempreture in 
T_Celsius = 23.4

# Time interval
t = 0.1

# Viscosity
eta = 9.2e-4

# Pixel Size
pixel_size = 314.3e-9

positions = multi_brownian_motion(num_particles, num_steps, r_list, (1, 1999), t, T_Celsius, eta, pixel_size)

## Background Generation

In [20]:
noiseimg = Image.open('noisesample.png')
img_gray = noiseimg.convert('L')
img_array = np.array(img_gray)
noisesample = img_array.flatten()
def generate_white_noise_image(resolution):
    array = random.choices(noisesample, k=resolution[1]*resolution[0])
    background = Image.fromarray(np.array(array).reshape(resolution[0], resolution[1]), 'L')
    return background

## Apply Brightness In the Center

In [21]:
def brighten_center(image, enhancement_factor=1.2, radius_factor=0.4):
    width, height = image.size

    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)

    left_up_point = (width * (0.5 - radius_factor), height * (0.5 - radius_factor))
    right_down_point = (width * (0.5 + radius_factor), height * (0.5 + radius_factor))
    draw.ellipse([left_up_point, right_down_point], fill=255)

    mask = mask.filter(ImageFilter.GaussianBlur(radius=int(width * 0.1)))

    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(0.8)
    brightened = enhancer.enhance(enhancement_factor)
    image.paste(brightened, (0,0), mask=mask)
    return image

## Image Generation

In [22]:
current_date_and_time = datetime.now()
folder = current_date_and_time.strftime('%Y-%m-%d-%H-%M-%S')
os.makedirs(f'output/{folder}', exist_ok=True)
os.makedirs(f'output/{folder}/Mask', exist_ok=True)
for i in range(num_steps):
    img = generate_white_noise_image(resolution)
    unetmask = Image.new('L', resolution, color="black")
    for idx, (x, y) in enumerate(positions[i]):
        transp_img = Image.new('L', resolution, color=0)
        mask = Image.new('L', resolution, color=0)

        m_draw = ImageDraw.Draw(mask)
        draw = ImageDraw.Draw(unetmask)
        
        r = radius[idx]
        left_up_point = (x-r, y-r)
        right_down_point = (x+r, y+r)
        
        draw.ellipse([left_up_point, right_down_point], fill=255)

        m_draw.ellipse([left_up_point, right_down_point], fill=255)
        blurred_circle = mask.filter(ImageFilter.GaussianBlur(radius=random.randint(1, 3)))
        img.paste(255, (0, 0), blurred_circle)
    img = brighten_center(img)

    filename = f"{i+1}.png"
    img.save(f'output/{folder}/{filename}')
    unetmask.save(f'output/{folder}/Mask/{filename}')
    print(f"Image saved as {'output/' + folder + '/' + filename}")

Image saved as output/2023-10-25-01-54-40/1.png
Image saved as output/2023-10-25-01-54-40/2.png
Image saved as output/2023-10-25-01-54-40/3.png
Image saved as output/2023-10-25-01-54-40/4.png
Image saved as output/2023-10-25-01-54-40/5.png
Image saved as output/2023-10-25-01-54-40/6.png
Image saved as output/2023-10-25-01-54-40/7.png
Image saved as output/2023-10-25-01-54-40/8.png
Image saved as output/2023-10-25-01-54-40/9.png
Image saved as output/2023-10-25-01-54-40/10.png
Image saved as output/2023-10-25-01-54-40/11.png
Image saved as output/2023-10-25-01-54-40/12.png
Image saved as output/2023-10-25-01-54-40/13.png
Image saved as output/2023-10-25-01-54-40/14.png
Image saved as output/2023-10-25-01-54-40/15.png
Image saved as output/2023-10-25-01-54-40/16.png
Image saved as output/2023-10-25-01-54-40/17.png
Image saved as output/2023-10-25-01-54-40/18.png
Image saved as output/2023-10-25-01-54-40/19.png
Image saved as output/2023-10-25-01-54-40/20.png
Image saved as output/2023-10

## Save Coordinates

In [23]:
dic = {}
for i in range(num_steps):
    l = []
    for j in range(num_particles):
        l.append(positions[i][j])
    dic[f'Step {i+1}'] = l

df = pd.DataFrame.from_dict(dic, orient='index').T

In [24]:
df.to_csv(f'output/{folder}/{folder}_coordinates.csv')

In [25]:
df

,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6,Step 7,Step 8,Step 9,Step 10,...,Step 41,Step 42,Step 43,Step 44,Step 45,Step 46,Step 47,Step 48,Step 49,Step 50
0,"[1334.7705760270414, 581.9393708784103]","[1332.7857255982121, 583.7011851909584]","[1332.869450727038, 582.3464813724888]","[1334.6906547011222, 581.2691549953493]","[1331.1651283116917, 581.4301091754604]","[1324.6890686783865, 580.9058274493649]","[1326.8691028295002, 579.5280791640096]","[1324.6624201546456, 582.9127060707453]","[1320.6750740102502, 582.7788403009497]","[1317.4810821147128, 578.9704802963239]",...,"[1343.4815718038753, 584.0320188168157]","[1341.3066330756922, 581.4443055543125]","[1341.870107057327, 580.0223506260687]","[1338.802737214203, 577.2714864914547]","[1334.5143307544306, 574.2292925805626]","[1338.9234016824498, 578.4595645956102]","[1338.4378158943298, 578.4502098588796]","[1338.7531237865835, 580.1461785934994]","[1340.6152471409698, 578.837923615518]","[1343.70000170583, 577.774926981275]"
1,"[1380.8601742104065, 1605.8196881981416]","[1386.1528031512753, 1610.6991471430817]","[1388.3335987406192, 1609.7620575783064]","[1386.4384987584237, 1612.4176889348994]","[1384.214613050193, 1609.4269957261924]","[1382.157717434515, 1605.6184988641987]","[1384.5292547321565, 1608.4394420981644]","[1382.4308184909994, 1602.8590741070745]","[1382.4358386319204, 1595.4427980334765]","[1382.3402983479716, 1590.1202448250476]",...,"[1383.4096438949712, 1597.4298013956304]","[1384.5451258988082, 1592.738625836611]","[1388.634320923141, 1596.68630033344]","[1383.2470267449112, 1598.388008632387]","[1382.1332703715764, 1597.7647286292963]","[1384.0946992431843, 1600.2777424518658]","[1382.3890498567405, 1600.3569387848563]","[1377.8795927704289, 1606.7271334253526]","[1375.1737211529908, 1608.0912797332192]","[1375.6478174275385, 1609.268641935781]"
2,"[2.0369826182755353, 1990.9671174239006]","[-0.7639952531081415, 1988.4452668521153]","[0.6494850458695374, 1988.5218409417942]","[0.6912509032854213, 1985.9487580211328]","[2.878150972116297, 1985.6805181683458]","[2.6591384183582303, 1986.8109261303075]","[1.0880846866742437, 1986.8682718930872]","[-1.75358874907232, 1986.7805304306735]","[-0.9961375612980187, 1985.506666414431]","[-1.253744456880939, 1986.4072273629067]",...,"[13.69342148642578, 1995.421614738056]","[9.97471937047602, 1993.5735072107552]","[14.144082304830427, 1995.7468282113564]","[15.621063350824079, 2000.0959075252754]","[15.44960673882881, 1996.8739583603563]","[15.510018324288055, 1996.8915847190474]","[12.753531812434423, 2000.8979963777335]","[9.104134059783945, 2000.8627733341714]","[6.27762377215626, 2005.7260894511626]","[4.887939104747163, 2008.3545151079982]"
3,"[1600.438751145498, 37.37757046607527]","[1597.352320344503, 34.63228636400944]","[1596.7887811355952, 30.792685616633793]","[1599.3471131718995, 30.74910400072022]","[1603.4677333766417, 31.50576793741341]","[1606.35922140228, 28.85933919538075]","[1612.6181669855212, 32.44173203500718]","[1612.2141214875005, 31.249248116521013]","[1609.15034145573, 37.5483651188883]","[1609.3730621718928, 38.56133503140238]",...,"[1596.1415305672801, 38.1441105948332]","[1596.4069037869951, 41.93097447283298]","[1593.2570189187559, 41.39250650409777]","[1595.8763242369828, 40.67299584886241]","[1600.7617835651113, 43.35227466145231]","[1599.315827826265, 41.30869674841902]","[1601.9059894998495, 40.74074885411117]","[1601.120322966642, 42.93972295070748]","[1601.9338284131468, 45.994538161174]","[1600.611985778076, 47.91553277358962]"
4,"[1395.730374999861, 1094.9750165756043]","[1393.9234585368772, 1089.321420559877]","[1395.3813015939552, 1090.3686602468183]","[1397.0252180443097, 1093.288233048592]","[1400.6508698005173, 1088.758976631467]","[1403.8273080856227, 1084.588626315274]","[1403.1893515225208, 1085.0969815283459]","[1402.320176565957, 1082.9769159104173]","[1402.760450562076, 1085.08213092696]","[1405.3010862076, 1089.4104019494139]",...,"[1377.8335376477905, 1086.07530

## Turn series of images as video

In [26]:
import cv2
import os

def get_numeric_part(filename):
    return int(filename.split('.')[0])

def images_to_video(image_folder, video_name, fps=10):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")] 
    images = sorted(images, key=get_numeric_part)
    
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    h, w, layers = frame.shape
    size = (w, h)

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, size)  # 'DIVX' is a codec used to compress and decompress video files, you can use 'XVID' or 'MP4V' for .avi or .mp4 format respectively
    
    for image in images:
        img_path = os.path.join(image_folder, image)
        img = cv2.imread(img_path)
        out.write(img)
    
    out.release()


images_to_video(f'output/{folder}', f'output/{folder}/{folder}_output_video.mp4')